![](img/330-banner.png)

# Lecture 18: Introduction to natural language processing 

UBC 2022-23

Instructor: Varada Kolhatkar

In [1]:
## Imports

import os
import re
import string
import sys
import time
from collections import Counter, defaultdict

sys.path.append("code/.")

import IPython
import numpy as np
import numpy.random as npr
import pandas as pd
from comat import CooccurrenceMatrix
from IPython.display import HTML
from ipywidgets import interactive
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from preprocessing import MyPreprocessor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kvarada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<br><br>

### Announcements

- Final exam: December 15th at 7:00pm [PHRM 1101](https://learningspaces.ubc.ca/classrooms/phrm-1101)
- Homework 7 will be released on Tuesday will be due in a week.  
- There will be 2 more homework assignments after this. The schedule is posted on the website.  
- Thanks for filling out the midterm survey! I very much appreciate it ❤️!

## Learning objectives

- Broadly explain what is natural language processing (NLP). 
- Name some common NLP applications. 
- Explain the general idea of a vector space model.
- Explain the difference between different word representations: term-term co-occurrence matrix representation and Word2Vec representation.
- Describe the reasons and benefits of using pre-trained embeddings. 
- Load and use pre-trained word embeddings to find word similarities and analogies. 
- Demonstrate biases in embeddings and learn to watch out for such biases in pre-trained embeddings.
- Use word embeddings in text classification and document clustering using `spaCy`.
- Explain the general idea of topic modeling. 
- Describe the input and output of topic modeling. 
- Carry out basic text preprocessing using `spaCy`.   

<br><br>

## What is Natural Language Processing (NLP)?

- What should a search engine return when asked the following question? 

![](img/lexical_ambiguity.png)

<!-- <center> -->
<!-- <img src="img/lexical_ambiguity.png" width="1000" height="1000"> -->
<!-- </center> -->

### What is Natural Language Processing (NLP)?
#### How often do you search everyday? 

![](img/Google_search.png)

<!-- <center> -->
<!-- <img src="img/Google_search.png" width="900" height="900"> -->
<!-- </center> -->

### What is Natural Language Processing (NLP)?

![](img/WhatisNLP.png)

<!-- <center> -->
<!-- <img src="img/WhatisNLP.png" width="800" height="800"> -->
<!-- </center> -->    

### Everyday NLP applications

![](img/annotation-image.png)
<!-- <center> -->
<!-- <img src="img/annotation-image.png" height="1200" width="1200"> -->
<!-- </center> -->

### NLP in news 

Often you'll NLP in news. Some examples: 
- [How suicide prevention is getting a boost from artificial intelligence](https://abcnews.go.com/GMA/Wellness/suicide-prevention-boost-artificial-intelligence-exclusive/story?id=76541481)
- [Meet GPT-3. It Has Learned to Code (and Blog and Argue).](https://www.nytimes.com/2020/11/24/science/artificial-intelligence-ai-gpt3.html)
- [How Do You Know a Human Wrote This?](https://www.nytimes.com/2020/07/29/opinion/gpt-3-ai-automation.html)
- ...

### Why is NLP hard?

- Language is complex and subtle. 
- Language is ambiguous at different levels. 
- Language understanding involves common-sense knowledge and real-world reasoning.
- All the problems related to representation and reasoning in artificial intelligence arise in this domain. 

### Example: Lexical ambiguity

<br><br>

![](img/lexical_ambiguity.png)

<!-- <img src="img/lexical_ambiguity.png" width="1000" height="1000"> -->
    

### Example: Referential ambiguity
<br><br>

<!-- <img src="img/referential_ambiguity.png" width="1000" height="1000"> -->

![](img/referential_ambiguity.png)
    

### [Ambiguous news headlines](http://www.fun-with-words.com/ambiguous_headlines.html)

<blockquote>
PROSTITUTES APPEAL TO POPE
</blockquote>    

- **appeal to** means make a serious or urgent request or be attractive or interesting?

<blockquote>
KICKING BABY CONSIDERED TO BE HEALTHY    
</blockquote> 

- **kicking** is used as an adjective or a verb?

<blockquote>
MILK DRINKERS ARE TURNING TO POWDER
</blockquote>

- **turning** means becoming or take up?

### Overall goal

- Give you a quick introduction to you of this important field in artificial intelligence which extensively used machine learning.   

![](img/NLP_in_industry.png)

<!-- <center> -->
<!-- <img src="img/NLP_in_industry.png" width="900" height="800"> -->
<!-- </center> -->

Today's plan

- Word embeddings
- Topic modeling 
- Basic text preprocessing 

<br><br>

## Word embeddings 

- How can we represent word meaning? 
- The idea is to create **representations of words** so that distances between words in the feature space indicate relationships between them. 

![](img/t-SNE_word_embeddings.png)
<!-- <center>
<img src="img/t-SNE_word_embeddings.png" width="850" height="850">
</center>
 -->
 
(Attribution: [Jurafsky and Martin 3rd edition](https://web.stanford.edu/~jurafsky/slp3/))

### Why do we care about word representations?

- So far we have been talking about sentence or document representations. 
- Now we are going one step back and talking about word representations. 
- Word is a basic unit of text and in order to capture meaning of text it is useful to capture word meaning (e.g., in terms of relationships between words).  

### Word meaning 

- A favourite topic of philosophers for centuries. 
- An example from legal domain: [Are hockey gloves "gloves, mittens, mitts" or "articles of plastics"?](https://www.scc-csc.ca/case-dossier/info/sum-som-eng.aspx?cas=36258)

<blockquote>
Canada (A.G.) v. Igloo Vikski Inc. was a tariff code case that made its way to the SCC (Supreme Court of Canada). The case disputed the definition of hockey gloves as either "gloves, mittens, or mitts" or as "other articles of plastic."
</blockquote>

![](img/hockey_gloves_case.png)
<!-- <center>
<img src="img/hockey_gloves_case.png" width="500" height="500">
</center>
 -->

### Word meaning: ML and Natural Language Processing (NLP) view
- Modeling word meaning that allows us to 
    - draw useful inferences to solve meaning-related problems 
    - find relationship between words, e.g., which words are similar, which ones have positive or negative connotations
    

### Word similarity   

- Suppose you are carrying out sentiment analysis. 
- Consider the sentences below. 

> S1: **The movie was awesome!!** 

> S2: **The movie was amazing!!** 

> S3: **The movie was awful!!** 


- Here we would like to capture similarity between **awesome** and **amazing** in reference to sentiment analysis task. 

<br><br>

### Word representations

### Activity 1:  Brainstorm ways to represent words (~2 mins) 

- Suppose you are building a question answering system and you are given the following question and three candidate answers. 
- What kind of relationship between words would we like our representation to capture in order to arrive at the correct answer?  
    
<blockquote>       
<p style="font-size:30px"><b>Question:</b> How <b>tall</b> is Machu Picchu?</p>
    <p style="font-size:30px"><b>Candidate 1:</b> Machu Picchu is 13.164 degrees south of the equator.</p>    
<p style="font-size:30px"><b>Candidate 2:</b> The official height of Machu Picchu is 2,430 m.</p>
<p style="font-size:30px"><b>Candidate 3:</b> Machu Picchu is 80 kilometres (50 miles) northwest of Cusco.</p>    
</blockquote> 
    

### Need a representation that captures relationships between words.

- We will be looking at two such representations.  
    1. Sparse representation with **term-term co-occurrence matrix**
    2. Dense representation with **word2vec skip-gram model**

### Term-term co-occurrence matrix

- So far we have been talking about documents and we created document-term co-occurrence matrix (e.g., bag-of-words representation of text). 
- We can also do this with words. The idea is to go through a corpus of text, keeping a count of all of the words that appear in context of each word (within a window).

- An example: 

<!-- <center>
<img src="img/term-term_comat.png" width="600" height="600">
</center>
     -->
(Credit: Jurafsky and Martin 3rd edition)


### Visualizing word vectors and similarity 

![](img/word_vectors_and_angles.png)
<!-- <center>
<img src="img/word_vectors_and_angles.png" width="800" height="800">
</center>
     -->
(Credit: Jurafsky and Martin 3rd edition)

- The similarity is calculated using dot products between word vectors.
    - Example: $\vec{\text{digital}}.\vec{\text{information}} = 0 \times 1 + 1\times 6 = 6$
    - Higher the dot product more similar the words.

### Visualizing word vectors and similarity

<img src="img/word_vectors_and_angles.png" width="600" height="600">

(Credit: Jurafsky and Martin 3rd edition)

- The similarity is calculated using dot products between word vectors.
    - Example: $\vec{\text{digital}}.\vec{\text{information}} = 0 \times 1 + 1\times 6 = 6$
    - Higher the dot product more similar the words.

- We can also calculate a normalized version of dot products. 
    $$similarity_{cosine}(w_1,w_2) = \frac{w_1.w_2}{\left\lVert w_1\right\rVert_2 \left\lVert w_2\right\rVert_2}$$


In [3]:
corpus = [
    "How tall is Machu Picchu?",
    "Machu Picchu is 13.164 degrees south of the equator.",
    "The official height of Machu Picchu is 2,430 m.",
    "Machu Picchu is 80 kilometres (50 miles) northwest of Cusco.",
    "It is 80 kilometres (50 miles) northwest of Cusco, on the crest of the mountain Machu Picchu, located about 2,430 metres (7,970 feet) above mean sea level, over 1,000 metres (3,300 ft) lower than Cusco, which has an elevation of 3,400 metres (11,200 ft).",
]
sents = MyPreprocessor(corpus)

In [4]:
cm = CooccurrenceMatrix(
    sents, window_size=2
)  # Let's build term-term co-occurrence matrix for our text.
comat = cm.fit_transform()
vocab = cm.get_feature_names()
df = pd.DataFrame(comat.todense(), columns=vocab, index=vocab, dtype=np.int8)
df.head()

,tall,machu,picchu,13.164,degrees,south,equator,official,height,"2,430",...,mean,sea,level,"1,000","3,300",ft,lower,elevation,"3,400","11,200"
tall,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
machu,1,0,5,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
picchu,1,5,0,1,1,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
13.164,0,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
degrees,0,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.metrics.pairwise import cosine_similarity


def print_cosine_similarity(df, word1, word2):
    """
    Returns similarity score between word1 and word2
    Arguments
    ---------
    df    -- (pandas.DataFrame)
        Dataframe containing word representations
    word1 -- (array)
        Representation of word1
    word2 -- (array)
        Representation of word2

    Returns
    --------
    None. Returns similarity score between word1 and word2 with the given representation
    """
    vec1 = df.loc[word1].to_numpy().reshape(1, -1)
    vec2 = df.loc[word2].to_numpy().reshape(1, -1)
    sim = cosine_similarity(vec1, vec2)
    print(
        "The dot product between %s and %s: %0.2f and cosine similarity is: %0.2f"
        % (word1, word2, np.dot(vec1.flatten(), vec2.flatten()), sim)
    )

In [6]:
print_cosine_similarity(df, "tall", "height")
print_cosine_similarity(df, "tall", "official")

The dot product between tall and height: 2.00 and cosine similarity is: 0.82
The dot product between tall and official: 1.00 and cosine similarity is: 0.50


- We are getting non-zero cosine similarity now and we are able to capture some similarities between words now. 
- That said similarities do not make much sense in the toy example above because we're using a tiny corpus. 
- To find meaningful patterns of similarities between words, we need a large corpus. 
- Let's try a bit larger corpus and check whether the similarities make sense. 

In [7]:
import wikipedia
from nltk.tokenize import sent_tokenize, word_tokenize

corpus = []

queries = [
    "Machu Picchu",
    "Sagarmatha",
    "Sequoia sempervirens",
    "President (country)",
    "Politics Canada",
]

for i in range(len(queries)):
    sents = sent_tokenize(wikipedia.page(queries[i]).content)
    corpus.extend(sents)
print("Number of sentences in the corpus: ", len(corpus))

Number of sentences in the corpus:  1470


In [8]:
sents = MyPreprocessor(corpus)
cm = CooccurrenceMatrix(sents)
comat = cm.fit_transform()
vocab = cm.get_feature_names()
df = pd.DataFrame(comat.todense(), columns=vocab, index=vocab, dtype=np.int8)
df

,machu,picchu,15th-century,inca,citadel,located,eastern,cordillera,southern,peru,...,canadian-politics.com,2008-08-28,comprehensive,cbc,digital,archives,boondoggles,elephants,campaigning,textbook
machu,0,91,1,6,2,2,1,0,1,5,...,0,0,0,0,0,0,0,0,0,0
picchu,91,0,1,7,4,1,1,0,1,4,...,0,0,0,0,0,0,0,0,0,0
15th-century,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
inca,6,7,1,0,1,1,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0
citadel,2,4,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
archives,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,2,0,1,1,1,0
boondoggles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
elephants,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,0,0,0
campaigning,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [9]:
print_cosine_similarity(df, "tall", "height")
print_cosine_similarity(df, "tall", "official")

The dot product between tall and height: 35.00 and cosine similarity is: 0.28
The dot product between tall and official: 1.00 and cosine similarity is: 0.02


<br><br><br><br>

## Dense representations 

- The goal is to learn general purpose embeddings that are useful for common tasks involving text data. 

### Sparse vs. dense word vectors
- Term-term co-occurrence matrix representation is long and sparse. 
    - length |V| is usually large (e.g., > 50,000) 
    - most elements are zero
- OK because there are efficient ways to deal with sparse matrices.

### Alternative 

- Learn short (~100 to 1000 dimensions) and dense vectors.  
- Short vectors are usually easier to train with ML models (less weights to train).
- They may generalize better.
- In practice they work much better! 

### Short and dense word embeddings 

<!-- ![](img/word2vec.png) -->
<center>
<img src="img/word2vec.png" width="400" height="400">
</center>    

### Pre-trained embeddings

- Training embeddings is computationally expensive
- For typical corpora, the vocabulary size is greater than 100,000.  
- If the size of embeddings is 300, the number of parameters of the model is $2 \times 30,000,000$. 
- So people have trained embeddings on huge corpora and made them available.  

A number of pre-trained word embeddings are available. The most popular ones are:  

- [word2vec](https://code.google.com/archive/p/word2vec/)
    * trained on several corpora using the word2vec algorithm 
- [wikipedia2vec](https://wikipedia2vec.github.io/wikipedia2vec/pretrained/)
    * pretrained embeddings for 12 languages 
- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using [the fastText algorithm](http://aclweb.org/anthology/Q17-1010)
    * published by Facebook    

Let's try Google News pre-trained embeddings.  

- You can download pre-trained embeddings from their original source. 
- `Gensim` provides an api to conveniently load them. 

### Dense word representations

- We won't really talk about how to create these embeddings.
    - If you are interested to know the details, I've [a video on Word2Vec](https://www.youtube.com/watch?v=rWoA-IKGDa8).  
- Below I am loading word vectors trained on Google News corpus using a package called `gensim`. 

```conda install -c anaconda gensim```

In [10]:
import gensim
import gensim.downloader as api

print(list(api.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [11]:
google_news_vectors = api.load('word2vec-google-news-300')

In [12]:
print("Size of vocabulary: ", len(google_news_vectors))

Size of vocabulary:  3000000


- `google_news_vectors` above has 300 dimensional word vectors for 3,000,000 unique words from Google news. 

### What can we do with these word vectors?

- Let's examine word vector for the word UBC. 

In [13]:
google_news_vectors["UBC"][:20]  # Representation of the word UBC

array([-0.3828125 , -0.18066406,  0.10644531,  0.4296875 ,  0.21582031,
       -0.10693359,  0.13476562, -0.08740234, -0.14648438, -0.09619141,
        0.02807617,  0.01409912, -0.12890625, -0.21972656, -0.41210938,
       -0.1875    , -0.11914062, -0.22851562,  0.19433594, -0.08642578],
      dtype=float32)

In [14]:
google_news_vectors["UBC"].shape

(300,)

Indeed it is a short and a dense vector! 

### Finding similar words 

- Given word $w$, search in the vector space for the word closest to $w$ as measured by cosine distance. 

In [15]:
google_news_vectors.most_similar("UBC")

[('UVic', 0.788647472858429),
 ('SFU', 0.7588527798652649),
 ('Simon_Fraser', 0.7356575131416321),
 ('UFV', 0.688043475151062),
 ('VIU', 0.6778583526611328),
 ('Kwantlen', 0.6771429181098938),
 ('UBCO', 0.6734487414360046),
 ('UPEI', 0.6731126308441162),
 ('UBC_Okanagan', 0.6709133982658386),
 ('Lakehead_University', 0.6622507572174072)]

In [16]:
google_news_vectors.most_similar("information")

[('info', 0.7363681793212891),
 ('infomation', 0.6800296306610107),
 ('infor_mation', 0.6733849048614502),
 ('informaiton', 0.6639009118080139),
 ('informa_tion', 0.6601256728172302),
 ('informationon', 0.633933424949646),
 ('informationabout', 0.6320979595184326),
 ('Information', 0.6186580061912537),
 ('informaion', 0.6093292236328125),
 ('details', 0.6063088774681091)]

If you want to extract all documents containing words similar to **information**, you could use this information.  

Google News embeddings also support multi-word phrases. 

In [17]:
google_news_vectors.most_similar("british_columbia")

[('alberta', 0.6111123561859131),
 ('canadian', 0.6086405515670776),
 ('ontario', 0.6031432151794434),
 ('erik', 0.5993570685386658),
 ('dominican_republic', 0.5925410985946655),
 ('costco', 0.5824530720710754),
 ('rhode_island', 0.5804311633110046),
 ('dreampharmaceuticals', 0.5755444169044495),
 ('canada', 0.5630921721458435),
 ('austin', 0.5623062252998352)]

### Finding similarity scores between words

In [18]:
google_news_vectors.similarity("Canada", "hockey")

0.27610135

In [19]:
google_news_vectors.similarity("Japan", "hockey")

0.0019627889

In [21]:
word_pairs = [
    ("height", "tall"),
    ("height", "official"),
    ("pineapple", "mango"),
    ("pineapple", "juice"),
    ("sun", "robot"),
    ("GPU", "hummus"),
]
for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], google_news_vectors.similarity(pair[0], pair[1]))
    )

The similarity between height and tall is 0.473
The similarity between height and official is 0.002
The similarity between pineapple and mango is 0.668
The similarity between pineapple and juice is 0.418
The similarity between sun and robot is 0.029
The similarity between GPU and hummus is 0.094


We are getting reasonable word similarity scores!! 

### Success of word2vec

- This analogy example often comes up when people talk about word2vec, which was used by the authors of this method. 
- **MAN : KING :: WOMAN : ?**
    - What is the word that is similar to **WOMAN** in the same sense as **KING** is similar to **MAN**?
- Perform a simple algebraic operations with the vector representation of words.
    $\vec{X} = \vec{\text{KING}} − \vec{\text{MAN}} + \vec{\text{WOMAN}}$
- Search in the vector space for the word closest to $\vec{X}$ measured by cosine distance.

![](img/word_analogies1.png)

<!-- <center>
<img src="img/word_analogies1.png" width="400" height="400">
</center>
     -->
(Credit: Mikolov et al. 2013)  

In [22]:
def analogy(word1, word2, word3, model=google_news_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [23]:
analogy("man", "king", "woman")

man : king :: woman : ?


,Analogy word,Score
0,queen,0.711819
1,monarch,0.618967
2,princess,0.590243
3,crown_prince,0.549946
4,prince,0.537732
5,kings,0.523684
6,Queen_Consort,0.523595
7,queens,0.518113
8,sultan,0.509859
9,monarchy,0.508741


In [27]:
analogy("Montreal", "Canadiens", "Vancouver")

Montreal : Canadiens :: Vancouver : ?


,Analogy word,Score
0,Canucks,0.821327
1,Vancouver_Canucks,0.750401
2,Calgary_Flames,0.705470
3,Leafs,0.695783
4,Maple_Leafs,0.691617
5,Thrashers,0.687504
6,Avs,0.681716
7,Sabres,0.665307
8,Blackhawks,0.664625
9,Habs,0.661023


In [28]:
analogy("Italy", "pizza", "Japan")

Italy : pizza :: Japan : ?


,Analogy word,Score
0,sushi,0.542754
1,ramen,0.541916
2,bento,0.539965
3,teriyaki,0.502705
4,yakisoba,0.502351
5,takoyaki,0.501550
6,onigiri,0.501540
7,burger,0.495270
8,ramen_noodle,0.493126
9,noodle,0.490753


In [29]:
analogy("USA", "pizza", "India") # Just for fun

USA : pizza :: India : ?


,Analogy word,Score
0,vada_pav,0.554463
1,jalebi,0.547090
2,idlis,0.540039
3,pav_bhaji,0.526046
4,dosas,0.521772
5,samosa,0.520700
6,idli,0.516858
7,pizzas,0.516199
8,tiffin,0.514347
9,chaat,0.508534


### Vada pav
![](img/vada_pav.jpeg)

<br><br>

So you can imagine these models being useful in many meaning-related tasks. 

![](img/word2vec-country-capitals.png)

(Credit: Mikolov et al. 2013)  

### Examples of semantic and syntactic relationships


<img src="img/word_analogies2.png" width="800" height="800">

(Credit: Mikolov 2013)

### Implicit biases and stereotypes in word embeddings

In [31]:
analogy("man", "computer_programmer", "woman")

man : computer_programmer :: woman : ?


,Analogy word,Score
0,homemaker,0.562712
1,housewife,0.510505
2,graphic_designer,0.505180
3,schoolteacher,0.497949
4,businesswoman,0.493489
5,paralegal,0.492551
6,registered_nurse,0.490797
7,saleswoman,0.488163
8,electrical_engineer,0.479773
9,mechanical_engineer,0.475540


![](img/eva-srsly.png)

- Embeddings reflect gender stereotypes present in broader society.
- They may also amplify these stereotypes because of their widespread usage. 
- See the paper [Man is to Computer Programmer as Woman is to ...](http://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf).

Most of the modern embeddings are de-biased for some obvious biases. 

```{note}
Note that these pre-trained word vectors are of big size (in several gigabytes). 
```

<br><br>

### Word vectors with spaCy

- spaCy also gives you access to word vectors with bigger models: `en_core_web_md` or `en_core_web_lr`
- `spaCy`'s pre-trained embeddings are trained on [OntoNotes corpus](https://catalog.ldc.upenn.edu/LDC2013T19).
- This corpus has a collection of different styles of texts such as telephone conversations, newswire, newsgroups, broadcast news, broadcast conversation, weblogs, religious texts. 
- Let's try it out. 

In [32]:
import spacy

nlp = spacy.load("en_core_web_md")

doc = nlp("pineapple")
doc.vector

array([ 6.5486e-01, -2.2584e+00,  6.2793e-02,  1.8801e+00,  2.0700e-01,
       -3.3299e+00, -9.6833e-01,  1.5131e+00, -3.7041e+00, -7.7749e-02,
        1.5029e+00, -1.7764e+00,  1.7324e+00,  1.6241e+00,  2.6455e-01,
       -3.0840e+00,  7.5715e-01, -1.2903e+00,  2.3571e+00, -3.8793e+00,
        7.7635e-01,  3.9372e+00,  3.9900e-01, -6.8284e-01, -1.4018e+00,
       -2.1673e+00, -1.9244e+00,  1.0629e+00,  3.3378e-01, -8.3864e-01,
       -2.5646e-01, -1.7198e+00, -5.4607e-02, -1.4614e+00,  1.3352e+00,
       -1.8177e+00,  1.7254e+00,  4.9624e-01,  1.1314e+00, -1.5295e+00,
       -8.8629e-01, -2.7562e-01,  7.1799e-01,  1.5554e-01,  3.4230e+00,
        2.7167e+00,  1.1793e+00,  2.0961e-01,  3.3121e-01,  1.2322e+00,
        1.4375e+00, -4.2099e-01,  6.2814e-01, -1.9051e+00,  3.0593e-02,
        6.1895e-01, -3.1495e-01, -2.0444e-04,  2.2073e+00,  3.8856e-01,
        1.6554e+00,  1.1932e+00,  2.6678e+00, -5.5454e-01, -1.2078e+00,
        1.5709e-01, -1.1324e+00, -2.0163e+00,  1.4567e+00, -2.42

<br><br>

### (Optional) Representing documents using word embeddings

- Assuming that we have reasonable representations of words. 
- How do we represent meaning of paragraphs or documents?
- Two simple approaches
    - Averaging embeddings
    - Concatenating embeddings

### Averaging embeddings

<blockquote>
All empty promises
</blockquote>
    
$(embedding(all) + embedding(empty) + embedding(promise))/3$

### Average embeddings with spaCy

- We can do this conveniently with [spaCy](https://spacy.io/usage/linguistic-features#vectors-similarity). 
- We need `en_core_web_md` model to access word vectors. 
- You can download the model by going to command line and in your course `conda` environment and download `en_core_web_md` as follows.   

```
conda activate cpsc330
python -m spacy download en_core_web_md
```

We can access word vectors for individual words in `spaCy` as follows. 

In [33]:
nlp("empty").vector[0:10]

array([ 0.010289,  4.9203  , -0.48081 ,  3.5738  , -2.2516  ,  2.1697  ,
       -1.0116  ,  2.4216  , -3.7343  ,  3.3025  ], dtype=float32)

We can get average embeddings for a sentence or a document in `spaCy` as follows: 

In [36]:
s = "All empty promises"
doc = nlp(s)
avg_sent_emb = doc.vector
print(avg_sent_emb.shape)
print("Vector for: {}\n{}".format((s), (avg_sent_emb[0:10])))

(300,)
Vector for: All empty promises
[-0.459937    1.9785299   1.0319      1.5123      1.4806334   2.73183
  1.204       1.1724668  -3.5227966  -0.05656664]


### Similarity between documents 

- We can also get similarity between documents as follows. 
- Note that this is based on average embeddings of each sentence. 

In [37]:
doc1 = nlp("Deep learning is very popular these days.")
doc2 = nlp("Machine learning is dominated by neural networks.")
doc3 = nlp("A home-made fresh bread with butter and cheese.")

# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))
print(doc2, "<->", doc3, doc2.similarity(doc3))

Deep learning is very popular these days. <-> Machine learning is dominated by neural networks. 0.6998688700400709
Machine learning is dominated by neural networks. <-> A home-made fresh bread with butter and cheese. 0.5098292369334257


- Do these scores make sense? 
- There are no common words, but we are still able to identify that doc1 and doc2 are more similar that doc2 and doc3.  
- You can use such average embedding representation in text classification tasks.

###  Airline sentiment analysis using average embedding representation 

- Let's try average embedding representation for airline sentiment analysis.
- We used this dataset last week so you should already have it in the data directory. If not you can download it [here](https://www.kaggle.com/jaskarancr/airline-sentiment-dataset). 

In [47]:
df = pd.read_csv("data/Airline-Sentiment-2-w-AA.csv", encoding="ISO-8859-1")

In [48]:
from sklearn.model_selection import cross_validate, train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)
X_train, y_train = train_df["text"], train_df["airline_sentiment"]
X_test, y_test = test_df["text"], test_df["airline_sentiment"]

In [49]:
train_df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
5789,681455792,False,finalized,3,2/25/15 4:21,negative,1.0,Can't Tell,0.6667,Southwest,NaN,mrssuperdimmock,NaN,0,@SouthwestAir link doesn't work,NaN,2/19/15 18:53,5.686040e+17,"Lake Arrowhead, CA",Pacific Time (US & Canada)
8918,681459957,False,finalized,3,2/25/15 9:45,neutral,1.0,NaN,NaN,Delta,NaN,labeles,NaN,0,@JetBlue okayyyy. But I had huge irons on way ...,NaN,2/17/15 10:18,5.677500e+17,NaN,NaN
11688,681462990,False,finalized,3,2/25/15 9:53,negative,1.0,Customer Service Issue,0.6727,US Airways,NaN,DropMeAnywhere,NaN,0,@USAirways They're all reservations numbers an...,"[0.0, 0.0]",2/17/15 14:50,5.678190e+17,"Here, There and Everywhere",Arizona
413,681448905,False,finalized,3,2/25/15 10:10,neutral,1.0,NaN,NaN,Virgin America,NaN,jsamaudio,NaN,0,@VirginAmerica no A's channel this year?,NaN,2/18/15 12:25,5.681440e+17,St. Francis (Calif.),Pacific Time (US & Canada)
4135,681454122,False,finalized,3,2/25/15 10:08,negative,1.0,Bad Flight,0.3544,United,NaN,CajunSQL,NaN,0,"@united missed it. Incoming on time, then Sat...",NaN,2/17/15 14:20,5.678110e+17,"Baton Rouge, LA",NaN


### Bag-of-words representation for sentiment analysis

In [50]:
pipe = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000)
)
pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe.named_steps["countvectorizer"].transform(X_train)
print("Data matrix shape:", X_train_transformed.shape)
pipe.fit(X_train, y_train);

Data matrix shape: (11712, 13064)


In [51]:
print("Train accuracy {:.2f}".format(pipe.score(X_train, y_train)))
print("Test accuracy {:.2f}".format(pipe.score(X_test, y_test)))

Train accuracy 0.94
Test accuracy 0.80


### Sentiment analysis with average embedding representation

- Let's see how can we get word vectors using `spaCy`. 
- Let's create average embedding representation for each example. 

In [52]:
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

We have reduced dimensionality from 13,064 to 300! 

In [53]:
X_train_embeddings.shape

(11712, 300)

In [54]:
X_train_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.259942,4.856640,-2.677500,-1.875390,0.459240,-0.304300,3.273020,2.008458,-4.818360,1.002320,...,1.555840,1.274170,0.716420,-1.296220,-1.403746,0.430054,1.107044,0.224080,-0.903256,-0.295254
1,-0.563826,0.869816,-2.462877,0.057751,0.892089,0.566698,-0.283121,3.594112,-1.578107,1.037976,...,0.270313,-0.948990,2.653996,-1.631425,-1.850329,0.432467,0.979464,1.015146,-2.604236,-0.188137
2,-0.707503,0.908782,-3.248327,-0.667797,2.590958,2.246804,-1.095754,3.953429,-1.524224,0.593270,...,0.948402,-1.760466,1.369070,-2.204359,-1.357214,0.389199,0.205336,0.100318,-2.950879,1.399726
3,-0.977736,4.676425,-0.224362,-1.011286,3.981441,-1.132660,0.988456,2.997113,0.553975,-0.539687,...,-0.229152,1.099266,1.652113,0.036738,0.315839,1.429018,0.557437,-0.333650,-0.281916,-0.587222
4,-0.725984,0.178514,-1.163662,0.597000,4.621603,-1.166608,1.256955,3.940082,0.530063,-0.816050,...,-0.398607,-0.219403,0.925384,-0.751906,-1.969212,1.381326,2.146303,0.195811,-2.914482,0.297692


### Sentiment classification using average embeddings 

- What are the train and test accuracies with average word embedding representation? 
- The accuracy is a bit better with less overfitting. 
- Note that we are using **transfer learning** here.
- The embeddings are trained on a completely different corpus. 

In [55]:
lgr = LogisticRegression(max_iter=1000)
lgr.fit(X_train_embeddings, y_train)
print("Train accuracy {:.2f}".format(lgr.score(X_train_embeddings, y_train)))
print("Test accuracy {:.2f}".format(lgr.score(X_test_embeddings, y_test)))

Train accuracy 0.80
Test accuracy 0.79


/Users/kvarada/opt/miniconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Sentiment classification using advanced sentence representations 

- Since, representing documents is so essential for text classification tasks, there are more advanced methods for document representation. 
- In homework 6, you also explore sentence embedding representation. 

In [56]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("paraphrase-distilroberta-base-v1")

In [57]:
emb_sents = embedder.encode("all empty promises")
emb_sents.shape

(768,)

In [58]:
emb_train = embedder.encode(train_df["text"].tolist())
emb_train_df = pd.DataFrame(emb_train, index=train_df.index)
emb_train_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
5789,-0.120494,0.250262,-0.022795,-0.116368,0.078649,0.037357,-0.251341,0.321429,-0.143985,-0.123487,...,0.199151,-0.150143,0.167078,-0.407671,-0.066161,0.049514,0.019384,-0.357601,0.125996,0.381074
8918,-0.182954,0.118282,0.066341,-0.136098,0.094947,-0.121303,0.069233,-0.097499,0.025739,-0.367980,...,0.113612,0.114662,0.049926,0.256736,-0.118687,-0.190720,0.011985,-0.141883,-0.230142,0.024899
11688,-0.032988,0.630251,-0.079515,0.148981,0.194709,-0.226263,-0.043630,0.217399,-0.010716,0.069644,...,0.676791,0.244483,0.051042,0.064099,-0.146945,0.090878,-0.090060,0.077212,-0.209226,0.308773
413,-0.119259,0.172168,0.098698,0.319858,0.415475,0.248360,-0.025923,0.385351,0.066414,-0.334289,...,-0.128482,-0.232447,-0.077805,0.181328,0.123243,-0.143694,0.660456,-0.048714,0.204774,0.163497
4135,0.094240,0.360193,0.213747,0.363690,0.275521,0.134936,-0.276319,0.009336,-0.021523,-0.258992,...,0.474885,0.242125,0.294532,0.279013,0.037831,0.089761,-0.548748,-0.049258,0.154525,0.141268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5218,-0.204409,-0.145290,-0.064201,0.213571,-0.140226,0.338555,-0.148578,0.224516,-0.042963,0.075930,...,-0.161949,0.040582,0.003971,-0.152549,-0.582908,-0.126527,0.060502,-0.111495,-0.097493,0.199321
12252,0.108408,0.438293,0.216812,-0.349289,0.422689,0.377760,0.045197,-0.034096,0.427570,-0.328272,...,0.257849,-0.032362,-0.275003,0.080452,-0.078975,-0.049972,-0.009762,-0.314754,-0.020774,0.268777
1346,0.068411,0.017591,0.236154,0.221447,-0.103567,0.055509,0.062909,0.067424,-0.003504,-0.157758,...,0.007711,0.323297,0.334637,0.367041,-0.068821,0.063667,-0.329991,0.232331,-0.184768,-0.000682
11646,-0.091488,-0.155708,0.032391,0.018313,0.524998,0.563933,-0.080984,0.097983,-0.535285,-0.377195,...,0.428014,-0.144573,0.045297,-0.107935,-0.135673,-0.290019,-0.137200,-0.503395,-0.042567,-0.282591


In [59]:
emb_test = embedder.encode(test_df["text"].tolist())
emb_test_df = pd.DataFrame(emb_test, index=test_df.index)
emb_test_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
1671,-0.002864,0.217326,0.124349,-0.082548,0.709687,-0.582442,0.257897,0.169356,0.248880,-0.266686,...,0.501767,0.095387,0.340172,0.087452,-0.368359,0.276195,0.238676,-0.219546,0.066603,0.256149
10951,-0.141048,0.137934,0.131319,0.194774,0.868205,0.078791,-0.131657,0.036243,-0.215749,-0.291946,...,-0.056255,-0.056040,0.147340,0.189665,-0.357366,0.061799,-0.161922,-0.278955,-0.173722,0.065324
5382,-0.252943,0.527507,-0.065608,0.013467,0.207989,0.003881,-0.066282,0.253166,0.021039,0.290957,...,0.180685,-0.042605,-0.173794,-0.079129,-0.169160,0.001317,-0.142593,-0.070816,-0.208826,0.400736
3954,0.054318,0.096738,0.113037,0.032039,0.493064,-0.641102,0.078760,0.402187,0.189743,-0.089538,...,0.123879,-0.285018,-0.297771,0.557171,0.076169,-0.029826,-0.076095,0.225454,0.002135,0.235430
11193,-0.065858,0.223270,0.507333,0.266193,0.104696,-0.219555,0.146247,0.315650,-0.126192,-0.435462,...,0.163994,0.207813,-0.001871,0.109391,-0.166778,-0.249200,-0.525419,-0.413066,0.119940,0.064297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5861,0.077512,0.322276,0.026697,-0.111392,0.174208,0.235201,0.053888,0.244941,0.181625,-0.226870,...,0.149843,0.311338,0.045975,-0.572319,-0.068256,0.217745,-0.056510,-0.355174,-0.028610,0.090676
3627,-0.173311,-0.023604,0.190388,-0.136543,-0.360269,-0.444687,0.056311,0.291941,-0.399719,-0.167930,...,0.042209,-0.161905,-0.040535,-0.050515,-0.252020,-0.133980,0.155001,-0.154482,-0.060201,-0.126556
12559,-0.124635,-0.101799,0.129061,0.636908,0.681090,0.399300,-0.078322,0.221823,-0.277218,-0.178589,...,0.022364,-0.109275,-0.073540,-0.153336,-0.123705,-0.238896,0.296447,-0.116797,0.115076,-0.345924
8123,0.063508,0.332506,0.119605,-0.001362,-0.161801,-0.082302,-0.025883,0.048027,0.126974,-0.159801,...,0.002221,-0.093885,0.430284,-0.088561,0.321488,0.447437,0.292395,-0.188566,-0.272767,0.126173


In [60]:
lgr = LogisticRegression(max_iter=1000)
lgr.fit(emb_train, y_train)
print("Train accuracy {:.2f}".format(lgr.score(emb_train, y_train)))
print("Test accuracy {:.2f}".format(lgr.score(emb_test, y_test)))

Train accuracy 0.87
Test accuracy 0.83


- Some improvement over bag of words and average embedding representations! 
- But much slower ...

## Break (5 min)

![](img/eva-coffee.png)

<br><br><br><br>

## Topic modeling 

### Topic modeling motivation

- Suppose you have a large collection of documents on a variety of topics. 

### Example: A corpus of news articles 

![](img/TM_NYT_articles.png)

<!-- <center> -->
<!-- <img src="img/TM_NYT_articles.png" height="2000" width="2000">  -->
<!-- </center> -->

### Example: A corpus of food magazines 

![](img/TM_food_magazines.png)

<!-- <center> -->
<!-- <img src="img/TM_food_magazines.png" height="2000" width="2000">  -->
<!-- </center> -->

### A corpus of scientific articles

![](img/TM_science_articles.png)

<!-- <center> -->
<!-- <img src="img/TM_science_articles.png" height="2000" width="2000">  -->
<!-- </center> -->

(Credit: [Dave Blei's presentation](http://www.cs.columbia.edu/~blei/talks/Blei_Science_2008.pdf))

### Topic modeling motivation

- Humans are pretty good at reading and understanding a document and answering questions such as 
    - What is it about?  
    - Which documents is it related to?     
- But for a large collection of documents it would take years to read all documents and organize and categorize them so that they are easy to search.
- You need an automated way
    - to get an idea of what's going on in the data or 
    - to pull documents related to a certain topic

### Topic modeling

- Topic modeling gives you an ability to summarize the major themes in a large collection of documents (corpus). 
    - Example: The major themes in a collection of news articles could be 
        - **politics**
        - **entertainment**
        - **sports**
        - **technology**
        - ...
- A common tool to solve such problems is unsupervised ML methods.
- Given the hyperparameter $K$, the idea of topic modeling is to describe the data using $K$ "topics"

### Topic modeling: Input and output

- Input
    - A large collection of documents
    - A value for the hyperparameter $K$ (e.g., $K = 3$)
- Output
    1. Topic-words association 
        - For each topic, what words describe that topic? 
    2. Document-topics association
        - For each document, what topics are expressed by the document? 
    

### Topic modeling: Example

- Topic-words association 
    - For each topic, what words describe that topic?  
    - A topic is a mixture of words. 

![](img/topic_modeling_word_topics.png)

<!-- <center> -->
<!-- <img src="img/topic_modeling_word_topics.png" height="1000" width="1000">  -->
<!-- </center> -->    

### Topic modeling: Example

- Document-topics association 
    - For each document, what topics are expressed by the document?
    - A document is a mixture of topics. 
    
![](img/topic_modeling_doc_topics.png)

<!-- <center> -->    
<!-- <img src="img/topic_modeling_doc_topics.png" height="800" width="800">  -->
<!-- </center> -->    

### Topic modeling: Input and output

- Input
    - A large collection of documents
    - A value for the hyperparameter $K$ (e.g., $K = 3$)
- Output
    - For each topic, what words describe that topic?  
    - For each document, what topics are expressed by the document?

![](img/topic_modeling_output.png)

<!-- <center> -->
<!-- <img src="img/topic_modeling_output.png" height="800" width="800">  -->
<!-- </center> -->    

### Topic modeling: Some applications

- Topic modeling is a great EDA tool to get a sense of what's going on in a large corpus. 
- Some examples
    - If you want to pull documents related to a particular lawsuit. 
    - You want to examine people's sentiment towards a particular candidate and/or political party and so you want to pull tweets or Facebook posts related to election.   

Let's carry out topic modeling using a popular model called **Latent Dirichlet Allocation (LDA)**. 

### Topic modeling toy example

In [63]:
toy_df = pd.read_csv("data/toy_lda_data.csv")
toy_df

,doc_id,text
0,1,famous fashion model
1,2,fashion model pattern
2,3,fashion model probabilistic topic model confer...
3,4,famous fashion model
4,5,fresh fashion model
5,6,famous fashion model
6,7,famous fashion model
7,8,famous fashion model
8,9,famous fashion model
9,10,creative fashion model


In [64]:
from gensim import corpora, models

corpus = [doc.split() for doc in toy_df["text"].tolist()]
# Create a vocabulary for the lda model
dictionary = corpora.Dictionary(corpus)
# Convert our corpus into document-term matrix for Lda
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

In [65]:
from gensim.models import LdaModel

# Train an lda model
lda = models.LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=3,
    random_state=123,
    passes=10,
)

In [66]:
### Examine the topics in our LDA model
lda.print_topics(num_words=4)

[(0,
  '0.303*"model" + 0.296*"probabilistic" + 0.261*"topic" + 0.040*"pattern"'),
 (1, '0.245*"kiwi" + 0.219*"apple" + 0.218*"nutrition" + 0.140*"health"'),
 (2, '0.308*"fashion" + 0.307*"model" + 0.180*"famous" + 0.071*"conference"')]

- The first entry in each tuple is the topic id. 
- The second entry has the most probable words in that topic along with their weights. 
- Here is how we can interpret the topics
    - Topic 0 $\rightarrow$ ML modeling 
    - Topic 1 $\rightarrow$ fruit and nutrition
    - Topic 2 $\rightarrow$ fashion

In [67]:
### Examine the topic distribution for a document
print("Document: ", corpus[0])
df = pd.DataFrame(lda[doc_term_matrix[0]], columns=["topic id", "probability"])
df.sort_values("probability", ascending=False)

Document:  ['famous', 'fashion', 'model']


,topic id,probability
2,2,0.828760
0,0,0.087849
1,1,0.083391


This document is made up of 
- ~83% topic 2
- ~9% topic 0 
- ~8% topic 1.  

You can also visualize the topics using `pyLDAvis`. 

```
pip install pyLDAvis

```

> Do not install it using `conda`. They have made some changes in the recent version and `conda` build is not available for this version yet. 

In [70]:
# Visualize the topics
import pyLDAvis

pyLDAvis.enable_notebook()
import pyLDAvis.gensim_models as gensimvis

vis = gensimvis.prepare(lda, doc_term_matrix, dictionary, sort_topics=False)
vis

/Users/kvarada/opt/miniconda3/envs/cpsc330/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x        y  topics  cluster       Freq
topic                                               
0     -0.187963  0.14888       1        1  40.126798
1      0.312197  0.02174       2        1  27.321760
2     -0.124234 -0.17062       3        1  32.551442, topic_info=             Term       Freq      Total Category  logprob  loglift
1         fashion  13.000000  13.000000  Default  15.0000  15.0000
5   probabilistic  15.000000  15.000000  Default  14.0000  14.0000
10           kiwi   9.000000   9.000000  Default  13.0000  13.0000
6           topic  14.000000  14.000000  Default  12.0000  12.0000
9           apple   8.000000   8.000000  Default  11.0000  11.0000
11      nutrition   8.000000   8.000000  Default  10.0000  10.0000
2           model  28.000000  28.000000  Default   9.0000   9.0000
0          famous   8.000000   8.000000  Default   8.0000   8.0000
12         health   5.000000   5.000000  Default   7.0000   7.0000
4      conference   3.000000   3.000000  Default   6.0000   6.0000
7           fresh   3.000000   3.000000  Default   5.0000   5.0000
8        creative   2.000000   2.000000  Default   4.0000   4.0000
3         pattern   2.000000   2.000000  Default   3.0000   3.0000
14         markov   1.000000   1.000000  Default   2.0000   2.0000
13         hidden   1.000000   1.000000  Default   1.0000   1.0000
5   probabilistic  15.092771  15.951783   Topic1  -1.2168   0.8578
6           topic  13.299810  14.180935   Topic1  -1.3433   0.8490
3         pattern   2.026453   2.667380   Topic1  -3.2248   0.6383
14         markov   1.178971   1.781560   Topic1  -3.7664   0.5003
13         hidden   1.178932   1.781561   Topic1  -3.7665   0.5002
2           model  15.435340  28.440734   Topic1  -1.1944   0.3020
8        creative   0.296761   2.698750   Topic1  -5.1459  -1.2945
4      conference   0.338443   3.572459   Topic1  -5.0145  -1.4435
7           fresh   0.297904   3.610581   Topic1  -5.1420  -1.5817
12         health   0.296319   5.447480   Topic1  -5.1474  -1.9983
0          famous   0.297340   8.034055   Topic1  -5.1439  -2.3834
11      nutrition   0.297434   8.178356   Topic1  -5.1436  -2.4009
9           apple   0.296893   8.178381   Topic1  -5.1454  -2.4028
10           kiwi   0.298092   9.088638   Topic1  -5.1414  -2.5043
1         fashion   0.329571  13.387340   Topic1  -5.0410  -2.7911
10           kiwi   8.490942   9.088638   Topic2  -1.4077   1.2295
9           apple   7.582598   8.178381   Topic2  -1.5208   1.2218
11      nutrition   7.581552   8.178356   Topic2  -1.5210   1.2217
12         health   4.852179   5.447480   Topic2  -1.9673   1.1818
7           fresh   2.211217   3.610581   Topic2  -2.7532   0.8072
8        creative   1.223500   2.698750   Topic2  -3.3450   0.5064
13         hidden   0.304390   1.781561   Topic2  -4.7361  -0.4694
14         markov   0.304335   1.781560   Topic2  -4.7363  -0.4696
3         pattern   0.305402   2.667380   Topic2  -4.7328  -0.8697
4      conference   0.304452   3.572459   Topic2  -4.7359  -1.1650
0          famous   0.305357   8.034055   Topic2  -4.7330  -1.9725
1         fashion   0.306382  13.387340   Topic2  -4.7296  -2.4797
6           topic   0.308711  14.180935   Topic2  -4.7221  -2.5298
5   probabilistic   0.307847  15.951783   Topic2  -4.7249  -2.6502
2           model   0.309770  28.440734   Topic2  -4.7186  -3.2223
1         fashion  12.751387  13.387340   Topic3  -1.1762   1.0737
0          famous   7.431357   8.034055   Topic3  -1.7161   1.0444
4      conference   2.929564   3.572459   Topic3  -2.6470   0.9239
2           model  12.695624  28.440734   Topic3  -1.1806   0.3158
8        creative   1.178489   2.698750   Topic3  -3.5576   0.2938
7           fresh   1.101460   3.610581   Topic3  -3.6252  -0.0649
14         markov   0.298254   1.781560   Topic3  -4.9316  -0.6650
13         hidden   0.298239   1.781561   Topic3  -4.9317  -0.6650
3         pattern   0.335525   2.667380   Topic3  -4.8139  -0.9508
12     

## Topic modeling pipeline 

- Above we worked with toy data. In the real world, we usually need to preprocess the data before passing it to LDA. 
- Here are typical steps if you want to carry out topic modeling on real-world data. 
    - Preprocess your corpus. 
    - Train LDA using `Gensim`.
    - Interpret your topics.     

### Data

In [73]:
import wikipedia

queries = [
    "Artificial Intelligence",
    "unsupervised learning",
    "Supreme Court of Canada",
    "Peace, Order, and Good Government",
    "Canadian constitutional law",
    "ice hockey",
]
wiki_dict = {"wiki query": [], "text": []}
for i in range(len(queries)):
    wiki_dict["text"].append(wikipedia.page(queries[i]).content)
    wiki_dict["wiki query"].append(queries[i])

wiki_df = pd.DataFrame(wiki_dict)
wiki_df

,wiki query,text
0,Artificial Intelligence,Artificial intelligence (AI) is intelligence -...
1,unsupervised learning,Supervised learning (SL) is a machine learning...
2,Supreme Court of Canada,The Supreme Court of Canada (SCC; French: Cour...
3,"Peace, Order, and Good Government","In many Commonwealth jurisdictions, the phrase..."
4,Canadian constitutional law,Canadian constitutional law (French: droit con...
5,ice hockey,Ice hockey (or simply hockey) is a team sport ...


### Preprocessing the corpus 

- **Preprocessing is crucial!**
- Tokenization, converting text to lower case
- Removing punctuation and stopwords
- Discarding words with length < threshold or word frequency < threshold        
- Possibly lemmatization: Consider the lemmas instead of inflected forms. 
- Depending upon your application, restrict to specific part of speech;
    * For example, only consider nouns, verbs, and adjectives
    
We'll use [`spaCy`](https://spacy.io/) for preprocessing. Check out available token attributes [here](https://spacy.io/usage/rule-based-matching#adding-patterns-attributes). 

In [74]:
import spacy

nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

In [75]:
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [76]:
wiki_df["text_pp"] = [preprocess(text) for text in nlp.pipe(wiki_df["text"])]

In [90]:
wiki_df

,wiki query,text,text_pp
0,Artificial Intelligence,Artificial intelligence (AI) is intelligence -...,artificial intelligence intelligence perceive ...
1,unsupervised learning,Supervised learning (SL) is a machine learning...,supervised learning machine learning paradigm ...
2,Supreme Court of Canada,The Supreme Court of Canada (SCC; French: Cour...,supreme court canada scc french cour suprême c...
3,"Peace, Order, and Good Government","In many Commonwealth jurisdictions, the phrase...",commonwealth jurisdiction phrase peace order g...
4,Canadian constitutional law,Canadian constitutional law (French: droit con...,canadian constitutional law french droit const...
5,ice hockey,Ice hockey (or simply hockey) is a team sport ...,ice hockey hockey team sport play ice skate ic...


### Topic modeling with `sklearn`

- We are using `Gensim` LDA because it's more flexible. 
- But we can also train an LDA model with `sklearn`.

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(wiki_df["text_pp"])

In [92]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 3
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)
document_topics = lda.fit_transform(X)

In [93]:
print("lda.components_.shape: {}".format(lda.components_.shape))

lda.components_.shape: (3, 4040)


In [94]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names())

/Users/kvarada/opt/miniconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [95]:
import mglearn

mglearn.tools.print_topics(
    topics=range(3),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       
--------      --------      --------      
hockey        intelligence  court         
ice           artificial    law           
team          human         displaystyle  
player        machine       federal       
play          problem       justice       
league        use           provincial    
game          research      case          
penalty       system        algorithm     
puck          government    learning      
canada        power         supreme       




Bad pipe message: %s [b'[4R(\x9dz/\x8a\n ly\xfaI\x0b\xb6\xbfH {\xb2\x93\xc9\x8a8\xd7\x80a\xbe1\x8d"\xfcW\x8b\xdd\x05\xa2\x1cK\xd5\xdf\xffk\xe4\xde\'2\xe8\xa7\xef\x00"\x13\x01\x13\x03\x13\x02\xc0+\xc0/\xcc\xa9\xcc\xa8\xc0,\xc00\xc0\n\xc0\t\xc0\x13\xc0\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x01\x91\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\x0e\x00\x0c\x00\x1d\x00\x17\x00\x18\x00\x19\x01\x00\x01\x01\x00\x0b\x00\x02\x01\x00\x00#\x00\x00\x00\x10\x00\x0e\x00\x0c\x02h2\x08http/1.1\x00\x05']
Bad pipe message: %s [b'\xcd\xd5\xddt)\x1e$  w\xfeq\xbfY \x83\x9a\xbe >B\x91>\x99\x11VH\xd0\xd7]\x9c\x84`-\x02\xdfS|\x9a\xbe\xde\xfeus\xe4\xa3\xfc\x1c\x9f9\xf7\x00"\x13\x01\x13\x03\x13\x02\xc0+\xc0/\xcc\xa9\xcc\xa8\xc0,\xc00\xc0\n\xc0\t\xc0\x13\xc0\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x01\x91\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\x0e\x00\x0c\x00\x1d\x00\x17\x00\x18\x00\x19\x01\x00\x01\x01\x00\x0b\x00\x02\x01\x00\x00#\x00\x00\x00\x10\x00\x0e\x00\x0c\x02h2\x08http/1.1\x00\x05\x00\x05\x01\x00\x00\

<br><br><br><br>

### (Optional) Training LDA with [gensim](https://radimrehurek.com/gensim/models/ldamodel.html)

To train an LDA model with [gensim](https://radimrehurek.com/gensim/models/ldamodel.html), you need

- Document-term matrix 
- Dictionary (vocabulary)
- The number of topics ($K$): `num_topics`
- The number of passes: `passes`

#### `Gensim`'s `doc2bow`
- Let's first create a dictionary using [`corpora.Dictionary`](https://radimrehurek.com/gensim/corpora/dictionary.html). 

In [78]:
corpus = [doc.split() for doc in wiki_df["text_pp"].tolist()]
dictionary = corpora.Dictionary(corpus)  # Create a vocabulary for the lda model
pd.DataFrame(
    dictionary.token2id.keys(), index=dictionary.token2id.values(), columns=["Word"]
)

,Word
0,"""however"
1,...
2,"1,000"
3,1.6
4,"134,777"
...,...
4102,wphl
4103,wrist
4104,yale
4105,youth


#### `Gensim`'s `doc2bow`
- Now let's convert our corpus into document-term matrix for LDA using [`dictionary.doc2bow`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow).
- For each document, it stores the frequency of each token in the document in the format (token_id, frequency). 

In [79]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]
doc_term_matrix[1][:20]

[(68, 4),
 (79, 3),
 (85, 1),
 (91, 3),
 (92, 1),
 (99, 1),
 (100, 3),
 (107, 1),
 (137, 51),
 (146, 1),
 (156, 4),
 (158, 1),
 (159, 1),
 (175, 2),
 (176, 4),
 (178, 3),
 (197, 1),
 (200, 1),
 (208, 1),
 (210, 2)]

Now we are ready to train an LDA model. 

In [80]:
from gensim.models import LdaModel

num_topics = 3

lda = models.LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    passes=10,
)

#### Examine the topics and topic distribution for a document in our LDA model

In [81]:
lda.print_topics(num_words=4)  # Topics

[(0,
  '0.030*"court" + 0.013*"\\displaystyle" + 0.011*"law" + 0.010*"algorithm"'),
 (1, '0.028*"hockey" + 0.017*"ice" + 0.013*"team" + 0.013*"player"'),
 (2,
  '0.015*"intelligence" + 0.012*"artificial" + 0.008*"human" + 0.008*"machine"')]

In [82]:
print("Document: ", wiki_df.iloc[0][0])
print("Topic assignment for document: ", lda[doc_term_matrix[0]])  # Topic distribution

Document:  Artificial Intelligence
Topic assignment for document:  [(2, 0.9998799)]


#### Visualize topics

In [84]:
vis = gensimvis.prepare(lda, doc_term_matrix, dictionary, sort_topics=False)
vis

/Users/kvarada/opt/miniconda3/envs/cpsc330/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.052931  0.122365       1        1  19.751024
1     -0.172218 -0.027854       2        1  47.066826
2      0.119287 -0.094511       3        1  33.182150, topic_info=              Term        Freq       Total Category  logprob  loglift
3572        hockey  240.000000  240.000000  Default  30.0000  30.0000
499          court  119.000000  119.000000  Default  29.0000  29.0000
3588           ice  145.000000  145.000000  Default  28.0000  28.0000
1001  intelligence   93.000000   93.000000  Default  27.0000  27.0000
4013          team  115.000000  115.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
864           goal   23.849719   59.779052   Topic3  -5.5356   0.1843
1095         learn   21.406403   44.630725   Topic3  -5.6436   0.3684
719        example   18.920586   35.558273   Topic3  -5.7671   0.4722
536          datum   18.969532   40.450361   Topic3  -5.7645   0.3459
2052         world   17.353973   50.362680   Topic3  -5.8535   0.0377

[189 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
2076      1  0.982279  \displaystyle
86        1  0.158045            act
86        2  0.767648            act
86        3  0.090312            act
122       3  0.964816          agent
...     ...       ...            ...
2052      2  0.595679          world
2052      3  0.337552          world
2057      1  0.392798           year
2057      2  0.392798           year
2057      3  0.224456           year

[269 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

<br><br><br><br>

## Basic text preprocessing [[video](https://youtu.be/7W5Q8gzNPBc)]

### Introduction 
- Why do we need preprocessing?
    - Text data is unstructured and messy. 
    - We need to "normalize" it before we do anything interesting with it. 
- Example:     
    - **Lemma**: Same stem, same part-of-speech, roughly the same meaning
        - Vancouver's &rarr; Vancouver
        - computers &rarr; computer 
        - rising &rarr; rise, rose, rises    

### Tokenization

- Sentence segmentation
    - Split text into sentences
- Word tokenization 
    - Split sentences into words

### Tokenization: sentence segmentation

<blockquote>
MDS is a Master's program at UBC in British Columbia. MDS teaching team is truly multicultural!! Dr. George did his Ph.D. in Scotland. Dr. Timbers, Dr. Ostblom, Dr. Rodríguez-Arelis, and Dr. Kolhatkar did theirs in Canada. Dr. Gelbart did his PhD in the U.S.
</blockquote>

- How many sentences are there in this text? 

In [ ]:
### Let's do sentence segmentation on "."
text = (
    "MDS is a Master's program at UBC in British Columbia. "
    "MDS teaching team is truly multicultural!! "
    "Dr. George did his Ph.D. in Scotland. "
    "Dr. Timbers, Dr. Ostblom, Dr. Rodríguez-Arelis, and Dr. Kolhatkar did theirs in Canada. "
    "Dr. Gelbart did his PhD in the U.S."
)

print(text.split("."))

### Sentence segmentation

- In English, period (.) is quite ambiguous. (In Chinese, it is unambiguous.)
    - Abbreviations like Dr., U.S., Inc.  
    - Numbers like 60.44%, 0.98
- ! and ? are relatively ambiguous.
- How about writing regular expressions? 
- A common way is using off-the-shelf models for sentence segmentation. 

In [ ]:
### Let's try to do sentence segmentation using nltk
from nltk.tokenize import sent_tokenize

sent_tokenized = sent_tokenize(text)
print(sent_tokenized)

### Word tokenization

<blockquote>
MDS is a Master's program at UBC in British Columbia. 
</blockquote>

- How many words are there in this sentence?  
- Is whitespace a sufficient condition for a word boundary?

### Word tokenization 

<blockquote>
MDS is a Master's program at UBC in British Columbia. 
</blockquote>

- What's our definition of a word?
    - Should British Columbia be one word or two words? 
    - Should punctuation be considered a separate word?
    - What about the punctuations in `U.S.`?
    - What do we do with words like `Master's`?
- This process of identifying word boundaries is referred to as **tokenization**.
- You can use regex but better to do it with off-the-shelf ML models.  

In [ ]:
### Let's do word segmentation on white spaces
print("Splitting on whitespace: ", [sent.split() for sent in sent_tokenized])

### Let's try to do word segmentation using nltk
from nltk.tokenize import word_tokenize

word_tokenized = [word_tokenize(sent) for sent in sent_tokenized]
# This is similar to the input format of word2vec algorithm
print("\n\n\nTokenized: ", word_tokenized)

### Word segmentation 

For some languages you need much more sophisticated tokenizers. 
- For languages such as Chinese, there are no spaces between words.
    - [jieba](https://github.com/fxsjy/jieba) is a popular tokenizer for Chinese. 
- German doesn't separate compound words.
    * Example: _rindfleischetikettierungsüberwachungsaufgabenübertragungsgesetz_
    * (the law for the delegation of monitoring beef labeling)

### Types and tokens
- Usually in NLP, we talk about 
    - **Type** an element in the vocabulary
    - **Token** an instance of that type in running text 


### Exercise for you 

<blockquote>    
UBC is located in the beautiful province of British Columbia. It's very close 
to the U.S. border. You'll get to the USA border in about 45 mins by car.     
</blockquote>  

- Consider the example above. 
    - How many types? (task dependent)
    - How many tokens? 

### Other commonly used preprocessing steps

- Punctuation and stopword removal
- Stemming and lemmatization

### Punctuation and stopword removal

- The most frequently occurring words in English are not very useful in many NLP tasks.
    - Example: _the_ , _is_ , _a_ , and punctuation
- Probably not very informative in many tasks 

In [ ]:
# Let's use `nltk.stopwords`.
# Add punctuations to the list.
stop_words = list(set(stopwords.words("english")))
import string

punctuation = string.punctuation
stop_words += list(punctuation)
# stop_words.extend(['``','`','br','"',"”", "''", "'s"])
print(stop_words)

In [ ]:
### Get rid of stop words
preprocessed = []
for sent in word_tokenized:
    for token in sent:
        token = token.lower()
        if token not in stop_words:
            preprocessed.append(token)
print(preprocessed)

### Lemmatization 

- For many NLP tasks (e.g., web search) we want to ignore morphological differences between words
    - Example: If your search term is "studying for ML quiz" you might want to include pages containing "tips to study for an ML quiz" or "here is how I studied for my ML quiz"
- Lemmatization converts inflected forms into the base form. 

In [ ]:
import nltk

nltk.download("wordnet")

In [ ]:
# nltk has a lemmatizer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
print("Lemma of studying: ", lemmatizer.lemmatize("studying", "v"))
print("Lemma of studied: ", lemmatizer.lemmatize("studied", "v"))

### Stemming

- Has a similar purpose but it is a crude chopping of affixes 
    * _automates, automatic, automation_ all reduced to _automat_.
- Usually these reduced forms (stems) are not actual words themselves.  
- A popular stemming algorithm for English is PorterStemmer. 
- Beware that it can be aggressive sometimes.

In [ ]:
from nltk.stem.porter import PorterStemmer

text = (
    "UBC is located in the beautiful province of British Columbia... "
    "It's very close to the U.S. border."
)
ps = PorterStemmer()
tokenized = word_tokenize(text)
stemmed = [ps.stem(token) for token in tokenized]
print("Before stemming: ", text)
print("\n\nAfter stemming: ", " ".join(stemmed))

### Other tools for preprocessing 

- We used [Natural Language Processing Toolkit (nltk)](https://www.nltk.org/) above
- Many available tools    
- [spaCy](https://spacy.io/)

### [spaCy](https://spacy.io/)

- Industrial strength NLP library. 
- Lightweight, fast, and convenient to use. 
- spaCy does many things that we did above in one line of code! 
- Also has [multi-lingual](https://spacy.io/models/xx) support. 

In [ ]:
import spacy

# Load the model
nlp = spacy.load("en_core_web_md")
text = (
    "MDS is a Master's program at UBC in British Columbia. "
    "MDS teaching team is truly multicultural!! "
    "Dr. George did his Ph.D. in Scotland. "
    "Dr. Timbers, Dr. Ostblom, Dr. Rodríguez-Arelis, and Dr. Kolhatkar did theirs in Canada. "
    "Dr. Gelbart did his PhD in the U.S."
)

doc = nlp(text)

In [ ]:
# Accessing tokens
tokens = [token for token in doc]
print("\nTokens: ", tokens)

# Accessing lemma
lemmas = [token.lemma_ for token in doc]
print("\nLemmas: ", lemmas)

# Accessing pos
pos = [token.pos_ for token in doc]
print("\nPOS: ", pos)

### Other typical NLP tasks 
In order to understand text, we usually are interested in extracting information from text. Some common tasks in NLP pipeline are: 
- Part of speech tagging
    - Assigning part-of-speech tags to all words in a sentence.
- Named entity recognition
    - Labelling named “real-world” objects, like persons, companies or locations.    
- Coreference resolution
    - Deciding whether two strings (e.g., UBC vs University of British Columbia) refer to the same entity
- Dependency parsing
    - Representing grammatical structure of a sentence

### Extracting named-entities using spaCy

In [ ]:
from spacy import displacy

doc = nlp(
    "University of British Columbia "
    "is located in the beautiful "
    "province of British Columbia."
)
displacy.render(doc, style="ent")
# Text and label of named entity span
print("Named entities:\n", [(ent.text, ent.label_) for ent in doc.ents])
print("\nORG means: ", spacy.explain("ORG"))
print("GPE means: ", spacy.explain("GPE"))

### Dependency parsing using spaCy

In [ ]:
doc = nlp("I like cats")
displacy.render(doc, style="dep")

### Many other things possible

- A powerful tool 
- All my Capstone groups last year used this tool. 
- You can build your own rule-based searches. 
- You can also access word vectors using spaCy with bigger models. (Currently we are using `en_core_web_md` model.)

<br><br><br><br>

## Summary

- NLP is a big and very active field. 
- We broadly explored three topics: 
    - Word embeddings using pretrained models 
    - Topic modeling 
    - Basic text preprocessing

Here are some resources if you want to get into NLP.


- The first resource I would recommend is the following book by Jurafsky and Martin. It's very approachable and fun. And the current edition is available online. 
    - [Speech and Language Processing by Dan Jurafsky and James H. Martin](https://web.stanford.edu/~jurafsky/slp3/)
- There is a course taught at Stanford called ["From languages to Information"](http://web.stanford.edu/class/cs124/) by one of the co-authors of the above book, and it might be a good introduction to NLP for you. Most of the [course material](https://www.youtube.com/playlist?list=PLoROMvodv4rOFZnDyrlW3-nI7tMLtmiJZ&disable_polymer=true) and [videos]() are available for free. 
- If you are into deep learning, you may refer to [this course](https://cs224d.stanford.edu/). Again, all lecture videos are available on youtube. 
- If you want to look at current advancements in the field, you'll find all NLP related publications [here](https://www.aclweb.org/anthology/). 